In [1]:

!pip install transformers datasets rouge-score tqdm

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=f00a846b9087ecd8d52af54e2c87d92739c1e013f8cf12de79023b37d89ae84f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

# Tải dataset từ Huggingface
dataset = load_dataset('Quoc59/Eval_QA_Wood')
base_model = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"
# Tải mô hình và tokenizer
model_name = "Quoc59/llama-3.2-3b-ChatBot-woodproduct"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

Eval.csv:   0%|          | 0.00/87.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/424 [00:00<?, ? examples/s]

config.json:   0%|          | 0.00/937 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/176 [00:00<?, ?B/s]

In [4]:
from rouge_score import rouge_scorer
import torch
from tqdm import tqdm  
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Khởi tạo scorer ROUGE
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)

# Hàm để tính ROUGE-L
def calculate_rouge_l(predictions, references):
    scores = []
    for pred, ref in zip(predictions, references):
        score = scorer.score(pred, ref)['rougeL']
        scores.append(score.fmeasure)
    return sum(scores) / len(scores)

# Tạo các dự đoán và tham chiếu
predictions = []
references = []

for example in tqdm(dataset['train'], desc="Processing"):  # Hoặc 'test' tùy vào tập dữ liệu
    question = example['Question']  # Truy cập trường Question
    answer = example['Answer']      # Truy cập trường Answer   # Truy cập trường response
    
    # Tokenize instruction và tạo dự đoán
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(**inputs, max_length = 50)  # Tăng max_length
    prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    predictions.append(prediction)
    references.append(answer)
    #print(predictions)
    #print(references)

# Tính ROUGE-L

rouge_l_score = calculate_rouge_l(predictions, references)
print(f"ROUGE-L Score: {rouge_l_score}")

Processing: 100%|██████████| 424/424 [09:18<00:00,  1.32s/it]


ROUGE-L Score: 0.29075608555761717
